<a href="https://colab.research.google.com/github/LC1332/Luotuo-Text-Embedding/blob/main/notebook/Luotuo_Embedding_Visualization_Medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Luotuo Embedding 骆驼嵌入: Generative Text Embedding Model distilled from OpenAI API

骆驼嵌入是一个文本嵌入(text embedding)模型，由冷子昂, 刘思诣, 黄泓森, 陈舒年, 胡婧, 孙骜, 陈启源, 李鲁鲁等开发

<details>
  <summary> 每个作者都是第一作者，顺序是随机的。(点这里具体)</summary>

李鲁鲁发起了项目，并完成了初步的验证，提出了KL散度Loss和Hard Negative挖掘。

刘思诣完成了初步训练框架的编写，以及支撑了后面模型上传到hugging face管线。

冷子昂完成了完整的大模型和小模型的训练，包括载入数据和损失函数的实现。

陈启源准备了CNewSum的数据，做了句子切分。

黄泓森负责爬取了OpenAI Embedding的数据。

陈舒年完成了重要的几个可视化。

孙骜（即将）用我们的得到的Embedding，完成CoT的提升实验。

胡婧收集了周杰伦的歌词，并（即将）完成更多的定量实验。

</details>

这个脚本中的Luotuo-Bert-Medium是由 胡婧@清华大学 训练的

这个脚本由李鲁鲁编写，并且尝试使用T4 GPU进行运行

骆驼嵌入是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一, 后者由李鲁鲁, 冷子昂, 陈启源发起。


## 安装环境

In [1]:
# Requirements
!pip -q install transformers
!pip -q install openai
!pip -q install openTSNE
!pip -q install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00


## 从骆驼嵌入项目中获取数据

In [2]:
!git clone https://github.com/LC1332/Luotuo-Text-Embedding.git

Cloning into 'Luotuo-Text-Embedding'...
remote: Enumerating objects: 657, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 657 (delta 30), reused 24 (delta 10), pack-reused 602
Receiving objects: 100% (657/657), 36.76 MiB | 17.66 MiB/s, done.
Resolving deltas: 100% (395/395), done.


## 从Hugging Face中获取模型

我们的骆驼BERT的主要模型为luotuo-bert

现在暂时是110M的小模型。之后中模型上线之后,luotuo-bert会替换为300M以上的较大的模型，同时小模型会更名为luotuo-bert-small

In [3]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
# Import our models. The package will take care of downloading the models automatically

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert-medium")
model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert-medium", trust_remote_code=True, model_args=model_args).to(device)

# tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
# model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
# model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True, model_args=model_args)

A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert-medium:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


## t-SNE 文本可视化实验

In [4]:
%cd Luotuo-Text-Embedding

/content/Luotuo-Text-Embedding


In [16]:
import pandas as pd
tsne_data_label_path = "/content/Luotuo-Text-Embedding/data/sentpair_label.csv"

data = pd.read_csv(tsne_data_label_path, header=0)
text_left = data["Column1"].tolist()
text_right = data["Column2"].tolist()
label = data["Label"].tolist()

In [17]:
import csv
import numpy as np
import sys
sys.path.append("..")


inputs = tokenizer(text_left, padding=True, truncation=True, return_tensors="pt").to(device)

# inputs = inputs

with torch.no_grad():
    embeddings_left = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
inputs = tokenizer(text_right, padding=True, truncation=True, return_tensors="pt").to(device)
with torch.no_grad():
    embeddings_right = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output

cos_sim_matrix = torch.matmul(embeddings_left, embeddings_right.t())
cos_sim_matrix /= torch.matmul(torch.norm(embeddings_left, dim=1, keepdim=True), torch.norm(embeddings_right, dim=1, keepdim=True).t())
tensor_cpu = cos_sim_matrix.cpu()

绘制tSNE可视化

In [19]:
# TODO: 这个不知道为什么挂掉了 回头fix一下

import sys
sys.path.append("..")
from lib.tsne import TSNE_Plot

merged_list = text_left + text_right
vis_label = label + label
merged_embed = torch.cat((embeddings_left.cpu(), embeddings_right.cpu()), dim=0)

# merged_embed = merged_embed.cpu()

# if the data have no labels, you can use the following code to cluster the data
tsne_plot = TSNE_Plot(merged_list, merged_embed, label = vis_label)
tsne_plot.tsne_plot(n_sentence=30)

## 相关性

In [ ]:
import ast

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
import lib.heatmap
import importlib
importlib.reload(sys.modules['lib.heatmap'])
from lib.heatmap import Heatmap

heat_plot_data_path = "/content/Luotuo-Text-Embedding/data/heatplot.csv"
heat_plot_data = pd.read_csv(heat_plot_data_path, header=0)
first = heat_plot_data["first"].tolist()
second = heat_plot_data["second"].tolist()
first_embed = heat_plot_data["first_embed"].apply(lambda x: ast.literal_eval(x)).tolist()
second_embed = heat_plot_data["second_embed"].apply(lambda x: ast.literal_eval(x)).tolist()
# positions = [(i, i) for i in range(0, 20, 2)] + [(1, 5), (2, 3), (15, 9), (5, 13), (17, 7)]
# heatmap = Heatmap(df, positions)
df = pd.DataFrame({ "first":first,
                    "second":second,
                    "first_embed":[np.array(first_embed[i]) for i in range(len(first_embed))],
                    "second_embed":[np.array(second_embed[i]) for i in range(len(second_embed))]})
heatmap = Heatmap(df)
heatmap.create_heatmap(font_path='./lib/arial.ttf')

## 模糊搜索

In [ ]:
import ast
import lib.fuzzySearch
import importlib

importlib.reload(lib.fuzzySearch)
from lib.fuzzySearch import FuzzySearch
fuzzy_search_data_path = "/content/Luotuo-Text-Embedding/data/search_data.csv"
stop_words_path = "/content/Luotuo-Text-Embedding/data/stop_word.txt"

fuzzy_search_data = pd.read_csv(fuzzy_search_data_path, header=0)
fuzzy_search_data["embed"] = fuzzy_search_data["embed"].apply(lambda x: ast.literal_eval(x))

In [ ]:
# fuzzy_search_data is the data source to search from, which is a pandas dataframe and should have two columns: sentence and embed
# sentence is the content of the sentence, each row is a string
# embed is the sentence embedding, each row is a list of float
fuzzy_search = FuzzySearch(fuzzy_search_data, stop_words_path = stop_words_path)

A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [ ]:
#虎扑报道马刺的保罗-加索尔与球队正式签订协议，有哪些相关的新闻？
fuzzy_search.search()

请输入您想搜索的问题，例如 '虎扑报道马刺的保罗-加索尔与球队正式签订协议，有哪些相关的新闻？'虎扑报道马刺的保罗-加索尔与球队正式签订协议，有哪些相关的新闻？


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.285 seconds.
DEBUG:jieba:Loading model cost 1.285 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


第1个结果：
<mark>虎扑</mark>体育7月14日讯。<mark>马刺</mark>官网宣布,<mark>保罗</mark>-加<mark>索尔</mark>与<mark>球队正式签订协议</mark>。根据之前的<mark>报道</mark>,加<mark>索尔</mark>与<mark>马刺签订</mark>的是一份为期2年3000万的<mark>协议</mark>。加<mark>索尔</mark>上赛季共打了72场比赛,场均31.8分钟,贡献16.5分11.0篮板4.1助攻2.0盖帽。

第2个结果：
2015-09-2809:03。新浪体育。显示图片下一站丹佛?北京时间9月28日,据雅<mark>虎</mark>体育<mark>报道</mark>,波特兰开拓者队已经与老将射手迈克-米勒达成了合同买断。一旦米勒成为自由球员之后,丹佛掘金队有意得到他。据雅<mark>虎</mark>体育得到的消息,如果迈克-米勒在规定的时间内没有被<mark>球队</mark>认领并成为一名自由球员,联盟里有一些<mark>球队</mark>将会对他感兴趣,其中就包括掘金队。目前,掘金队中已经有14个保障合同。事实上早在一年前,掘金队就在自由球员市场上猛追过米勒。但最终,米勒决定跟随勒布朗-詹姆斯加盟骑士队。今年7月,开拓者队在关于布兰登-海伍德的那笔交易中得到了迈克-米勒。作为2000年的NBA首轮5号新秀,迈克-米勒曾获得过2001年最佳新秀称号、2006年最佳第六人称号,并且曾跟随迈阿密热火队夺得过两次NBA总冠军。在长达15年的职业生涯里,米勒一直被视为一名关键时刻靠得住的射手,并且能够为争冠<mark>球队</mark>做出贡献。他的生涯场均数据是11.3分、4.4个篮板和2.7次助攻。

第3个结果：
<mark>虎扑</mark>体育9月3日讯。根据《太阳哨兵报》记者IraWinderman的<mark>报道</mark>,帕特-莱利本周接受采访时表示,交易得到沙奎尔-奥尼尔是他们队史上最大的交易收获。在2004年夏天,奥尼尔被湖人交易到了热火,随后他就与德维恩-韦德在2005-06赛季一起为热火拿到了他们队史上的首个总冠军。“我会这么说,我的意思也是这个,”莱利说,“获得奥尼尔比我们史上的任何收获都重要,其中也包括三巨头。”莱利称热火交易得到奥尼尔的重要

### 文本聚类
当语料label没有给定时，算法将自动聚类并绘制T-SNE plot

In [ ]:
import ast
tsne_data_cluster_path = "/content/Luotuo-Text-Embedding/data/search_data.csv"
tsne_data_cluster = pd.read_csv(tsne_data_cluster_path, header=0)
text = tsne_data_cluster["sentence"].tolist()
embed = tsne_data_cluster["embed"].apply(lambda x: ast.literal_eval(x))

tsne_plot_cluster = TSNE_Plot(text, embed, n_clusters=4)
tsne_plot_cluster.tsne_plot(n_sentence=40)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



# TODO:
* 模糊问题搜索
* 文本聚类
* 少样本分类学习